# (목적)Gradient를 확인한다
## 전체 Gradient에 3을 곱한것과 parameter 별로 gradient에 3을 곱한 것이 차이가 나는지 확인

In [24]:
import torch
import torch.nn as nn

import torch.nn.functional as F

# 예제 데이터 생성
X = torch.randn(100, 2)
y = torch.randint(0, 2, (100,))

# 모델 정의
class ClassificationModel(nn.Module):
    def __init__(self):
        super(ClassificationModel, self).__init__()
        self.fc1 = nn.Linear(2, 2)
        self.fc2 = nn.Linear(2, 2)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

model = ClassificationModel()

# Cross Entropy Loss와 KL Divergence Loss 정의
cross_entropy_loss = nn.CrossEntropyLoss()
kl_divergence_loss = nn.KLDivLoss(reduction='batchmean')

# Forward Pass
outputs = model(X)

# Cross Entropy Loss 계산
ce_loss = F.cross_entropy(outputs, y)

# KL Divergence Loss 계산
p = torch.tensor([0.1, 0.2])  # 예시 분포
kl_loss = kl_divergence_loss(torch.log(outputs), p)
# kl_loss = ce_loss

# Gradient 계산 (torch.autograd.grad 사용)
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
kl_gradient = torch.autograd.grad(kl_loss, model.parameters(), retain_graph=True)

# Gradient 확인
print("Cross Entropy Loss Gradient:")
for grad_param in ce_gradient:
    print(grad_param)

# print("\nKL Divergence Loss Gradient:")
# for grad_param in kl_gradient:
#     print(grad_param)

Cross Entropy Loss Gradient:
tensor([[-0.0053, -0.0007],
        [ 0.0170,  0.0024]])
tensor([ 0.0045, -0.0146])
tensor([[-0.0140, -0.0017],
        [ 0.0140,  0.0017]])
tensor([-0.0492,  0.0492])


In [25]:
ce_loss *= 2
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)

# Gradient 확인
print("Cross Entropy Loss Gradient:")
for grad_param in ce_gradient:
    print(grad_param)

Cross Entropy Loss Gradient:
tensor([[-0.0105, -0.0015],
        [ 0.0341,  0.0047]])
tensor([ 0.0090, -0.0293])
tensor([[-0.0281, -0.0034],
        [ 0.0281,  0.0034]])
tensor([-0.0985,  0.0985])


In [26]:
kl_loss

tensor(nan, grad_fn=<DivBackward0>)

In [27]:
total_loss = 1 * ce_loss +  2* kl_loss
total_loss
total_gradient = torch.autograd.grad(total_loss, model.parameters(), retain_graph=True)
print(total_gradient)

(tensor([[-0.5237, -0.3311],
        [ 0.3590,  0.2290]]), tensor([ 0.8400, -0.5175]), tensor([[-0.0887,  0.0450],
        [ 0.1690,  0.2797]]), tensor([0.4007, 1.1696]))


In [28]:
names_grads_copy = {}

for param_name, ce_grad, kl_grad in zip(model.parameters(), ce_gradient, kl_gradient):
    
    #kl_grad = None
    
    if not kl_grad == None:
        names_grads_copy[param_name] = torch.tensor(1) * ce_grad + torch.tensor(2) *  kl_grad
    else:
        names_grads_copy[param_name] = torch.tensor(1) * ce_grad 
    
for key, value in names_grads_copy.items():
    print(value)

tensor([[-0.5237, -0.3311],
        [ 0.3590,  0.2290]])
tensor([ 0.8400, -0.5175])
tensor([[-0.0887,  0.0450],
        [ 0.1690,  0.2797]])
tensor([0.4007, 1.1696])


## 1. Gradient를 추가적으로 구해도 retain_graph=True를 설정하면, 값이 변하지 않는다

In [29]:
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(grad_param)

tensor([[-0.0105, -0.0015],
        [ 0.0341,  0.0047]])
tensor([ 0.0090, -0.0293])
tensor([[-0.0281, -0.0034],
        [ 0.0281,  0.0034]])
tensor([-0.0985,  0.0985])


# 2. 전체 gradient에 3을 곱한다
## 1) 안된다

In [30]:
ce_gradient =  torch.tensor(3) * torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(grad_param)

tensor([[-0.0105, -0.0015],
        [ 0.0341,  0.0047]])
tensor([ 0.0090, -0.0293])
tensor([[-0.0281, -0.0034],
        [ 0.0281,  0.0034]])
tensor([-0.0985,  0.0985])
tensor([[-0.0105, -0.0015],
        [ 0.0341,  0.0047]])
tensor([ 0.0090, -0.0293])
tensor([[-0.0281, -0.0034],
        [ 0.0281,  0.0034]])
tensor([-0.0985,  0.0985])
tensor([[-0.0105, -0.0015],
        [ 0.0341,  0.0047]])
tensor([ 0.0090, -0.0293])
tensor([[-0.0281, -0.0034],
        [ 0.0281,  0.0034]])
tensor([-0.0985,  0.0985])


## 2) 덧셈 조차 하면 안된다

In [31]:
total_gradient = ce_gradient + kl_gradient
total_gradient

(tensor([[-0.0105, -0.0015],
         [ 0.0341,  0.0047]]),
 tensor([ 0.0090, -0.0293]),
 tensor([[-0.0281, -0.0034],
         [ 0.0281,  0.0034]]),
 tensor([-0.0985,  0.0985]),
 tensor([[-0.0105, -0.0015],
         [ 0.0341,  0.0047]]),
 tensor([ 0.0090, -0.0293]),
 tensor([[-0.0281, -0.0034],
         [ 0.0281,  0.0034]]),
 tensor([-0.0985,  0.0985]),
 tensor([[-0.0105, -0.0015],
         [ 0.0341,  0.0047]]),
 tensor([ 0.0090, -0.0293]),
 tensor([[-0.0281, -0.0034],
         [ 0.0281,  0.0034]]),
 tensor([-0.0985,  0.0985]),
 tensor([[-0.2566, -0.1648],
         [ 0.1624,  0.1121]]),
 tensor([ 0.4155, -0.2441]),
 tensor([[-0.0303,  0.0242],
         [ 0.0705,  0.1382]]),
 tensor([0.2496, 0.5356]))

## 3) loss에 3을 곱해줘야 내가 원하는 값을 얻을 수 있다

In [32]:
ce_gradient =  torch.autograd.grad(torch.tensor(3) * ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(grad_param)

tensor([[-0.0316, -0.0044],
        [ 0.1022,  0.0141]])
tensor([ 0.0271, -0.0878])
tensor([[-0.0842, -0.0102],
        [ 0.0842,  0.0102]])
tensor([-0.2954,  0.2954])


# 3. 각 parameter 별로 3을 곱해준다

In [33]:
# Gradient 확인
print("Cross Entropy Loss Gradient:")
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(torch.tensor(3) * grad_param)

Cross Entropy Loss Gradient:
tensor([[-0.0316, -0.0044],
        [ 0.1022,  0.0141]])
tensor([ 0.0271, -0.0878])
tensor([[-0.0842, -0.0102],
        [ 0.0842,  0.0102]])
tensor([-0.2954,  0.2954])


## 4. weight에 2를 곱하고 gradient를 구하면 값이 달라진다

In [34]:
# 각 레이어의 가중치에 2를 곱함
for param in model.parameters():
    param.data = param.data * 2

ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(grad_param)

tensor([[-0.0105, -0.0015],
        [ 0.0341,  0.0047]])
tensor([ 0.0090, -0.0293])
tensor([[-0.0281, -0.0034],
        [ 0.0281,  0.0034]])
tensor([-0.0985,  0.0985])


# 5. 각 parameter 별로 3을 더해준다

In [35]:
# Gradient 확인
print("Cross Entropy Loss Gradient:")
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(grad_param)

Cross Entropy Loss Gradient:
tensor([[-0.0105, -0.0015],
        [ 0.0341,  0.0047]])
tensor([ 0.0090, -0.0293])
tensor([[-0.0281, -0.0034],
        [ 0.0281,  0.0034]])
tensor([-0.0985,  0.0985])


In [36]:
# Gradient 확인
print("Cross Entropy Loss Gradient:")
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(torch.tensor(3) + grad_param)

Cross Entropy Loss Gradient:
tensor([[2.9895, 2.9985],
        [3.0341, 3.0047]])
tensor([3.0090, 2.9707])
tensor([[2.9719, 2.9966],
        [3.0281, 3.0034]])
tensor([2.9015, 3.0985])
